# Checkpoint 2

**Due: Friday, 6 November, 2020 at 5:00pm GMT**

### Read This First
1. Use the constants provided in the cell below. Do not use your own constants.

2. Put the code that produces the output for a given task in the cell indicated. You are welcome to add as many cells as you like for imports, function definitions, variables, etc. **Additional cells need to be in the proper order such that your code runs correctly the first time through.**

3. **IMPORTANT!** Before submitting your notebook, clear the output by clicking *Restart & Clear Output* from the *Kernel* menu. If you do not do this, the file size of your notebook will be very large.

General comments on my code:

I have learned my lesson from checkpoint 1 and I have decided that for each task reinitializing all the constants makes for much easier debugging and control of the code. This way I dont have search the whole notebook to see where is something defined and what is its value. This is also the reason why I decided not to make a lot of functions in my code, which would definitely make my code look nicer and neater, but would make debugging and controling my code harder. Thats why the same lines of code are seen in most tasks, but that made it easier for me to solve tasks, as I could easily see what exactly is going in my code.

For most of the tasks I implemented both the spectral and the explicit euler method of solving differential equations. The spectral method outperforms the explicit euler one, and thats the one I didnt comment out so it still runs. I used the explicit euler method as a form of sanity check, my reasoning was if both methods agreed on a result, the result is good and both method are implemented correctly. I left the code for explicit euler because it does give incredibly precise results, and if I ever need it for future refrencing, I have it right here.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import time

from scipy.sparse.linalg import eigsh
from scipy.sparse.linalg import spsolve
from scipy.sparse import diags
from scipy.sparse import identity

plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 16

# Task 1 (25p)

Find numerically the first 101 lowest eigenvalues (“energies”) of the discrete Hamiltonian matrix, H, for V=0 and x=0,...,L, with L=100, dx=1/8 and with reflecting boundary conditions:

$
\begin{align}
\psi_N \equiv \psi_{N-1},
\end{align}
$

$
\begin{align}
\psi_{-1} \equiv \psi_0,
\end{align}
$

Here the index -1 denotes the element to the left of the element zero, not the element N-1 as in Python. Note that indices $-1, N$ refer to elements of $\psi$ that are outside the matrix H.

Hint: The eigenvalues, $w_n$, should be

$
\large
\begin{align}
w_n = \frac{2\left(1 - \cos (\frac{n \pi}{N}) \right)}{dx^2}
\end{align}
$

for n=0, 1,..., 100. The corresponding (non-normalized) eigenvectors, $v_n$ are

$
\large
\begin{align}
v_n = \cos \left( \frac{n \pi x}{L} \right)
\end{align}
$

for x=0, dx,..., (N-1)dx.

In [ ]:
### Values for task 1
L  = 100
dx = 1/8
N  = int(L/dx)
print(f"Matrix size: {N}.")

In the cell below, compute the 101 lowest eigenvalues given the values of V, L, and dx from above. The function, `task1` should return an array of the eigenvalues.

It may be useful to write a generalized function for creating the Hamiltonian given values of N, dx, and V.

**Imposing boundary conditions**

The laplace matrix is modified so the $\psi_{-1} = \psi_{0}$ and $\psi_{N} = \psi_{N-1}$ conditions are satisfied. It is modified in a way such that the weight of the first and last term is changed, as if it was accounted for the values outside our vector. The modified matrix looks like
$
\begin{pmatrix}
-1 &  1 &  0 &  0 &  0 \\
 1 & -2 &  1 &  0 &  0 \\
 0 &  1 & -2 &  1 &  0 \\
 0 &  0 &  1 & -2 &  1 \\
 0 &  0 &  0 &  1 & -1
 \end{pmatrix}
 $
 
This works because if we imagine a matrix being bigger than the actuall real space we are looking at we would get that the first entry would be $\psi_{-1} -2\psi_{0} + \psi_{1}$. Sub in $\psi_{-1} = \psi_{0}$ and we get $-1\psi_{0} + \psi_{1}$ which is exactly what this matrix represents.

In [ ]:
def make_Laplacian(n, dr):
    """Creates a Laplacian with reflecting boundary conditions. 
    Parameters are: n - the size of the matrix and dr - the spacing between two sucessive discrete points on a real line"""
    
    # Create the Laplace operator
    diagOne = np.ones(n-1)
    diagZero = -2*np.ones(n)
    # Manually change
    diagZero[0] = -1
    diagZero[-1] = -1
    diagMinusOne = np.ones(n-1)
    laplace = -1/(dr**2) * diags([diagMinusOne, diagZero, diagOne], [-1,0,1])
    
    return laplace

In [ ]:
def task1():
    hamiltonian = make_Laplacian(N, dx)
    
    eigenvalues = eigsh(hamiltonian, k=101, return_eigenvectors=False, which='SA') # SA stands for smallest algebraic
    return eigenvalues[::-1] # Reverse the order of the eigenvalues because we want smallest ones to come first

## Testing task 1

The cell below will run your function and compare it with the expected values.

In [ ]:
t1 = time.time()
ww = task1()
t2 = time.time()
print (f"Time to solution: {t2-t1} seconds.")

my_n = np.arange(101)
ww_expected = 2 * (1 - np.cos(my_n*np.pi/N)) / dx**2

plt.plot(my_n, ww, label='Computed', alpha=0.8)
plt.plot(my_n, ww_expected, label='Expected', linewidth=2, linestyle='--', alpha=0.8)
plt.xlabel(r'$n$')
plt.ylabel(r'$w_{n}$')
plt.legend()
plt.show()

diff = np.abs(ww - ww_expected) / np.max(np.abs([ww, ww_expected]), axis=0)
print (f"Max difference is {diff[1:].max()}.")
assert (diff[1:] < 1e-6).all()

# Task 2 (5p)

Select dx as a negative power of two ($dx=2^{-n}$ for n>0) such that the 101th eigenvalue differs from the dx$\rightarrow$0 limiting value of

$
\begin{align}
w_{101,exact} = \frac{(100\pi/N)^{2}}{dx^2}
\end{align}
$

by less than 0.1%, i.e., $|w_{101}/w_{101,exact}-1|<0.001$.

Motivation: selecting a sufficiently small dx is required to obtain a good approximation to the original (continuous) equation (1) for subsequent tasks.

In the cell below, compute a value of dx that satisfies the above contraint. The function `task2` should return the value of dx that you have calculated.

In [ ]:
# This function is very efficient in calculating the eigenvalues of a specific form of matrices and is perfect for our needs
# In this checkpoint
from scipy.linalg import eigh_tridiagonal

In [ ]:
def get_101st_eigenvalue(N, dx):
    """This function finds the 101st eigenvalue of a discrete Laplacian matrix
    Parameters are: N - size of the matrix and dx - the spacing between two discrete points"""
    
    crtLaplacian = make_Laplacian(N, dx).toarray() # Has to be in full form for np.diagonal to work
    # Find only the 101st eigenvector which is at index 100
    lam, phi = eigh_tridiagonal(np.diagonal(crtLaplacian), np.diagonal(crtLaplacian, 1), select='i', select_range=(100, 100))
    return lam[0]

In [ ]:
def task2():
    L = 100
    # Starting numbers
    dx = 1/2
    N = int(L/dx)
    
    # Starting eigenvector values
    w101ex = (100*np.pi/N)**2/dx**2
    w101 = get_101st_eigenvalue(N, dx)
    
    # Iterate
    while abs(w101/w101ex - 1) > 0.001:
        # Update values
        dx = dx/2
        N = int(L/dx)
        w101ex = (100*np.pi/N)**2/dx**2
        w101 = get_101st_eigenvalue(N, dx)
        
    return dx

## Testing task 2

The cell below will run the `task2` function. We will verify that value of dx return satisfies the criterion outlined above.

In [ ]:
t1 = time.time()
my_dx = task2()
t2 = time.time()
print (f"Time to solution: {t2-t1} seconds.")


In [ ]:
print(my_dx)

# Task 3 (20p)

Solve equation (1) with the initial condition (2) (the Schroedinger equation and the Gaussian function from the checkpoint's description) for V(x)=0, on a domain x=0...100, for t=0...4 and dx from task 2. Plot $|\psi(x,t)|^2$ and determine its mean given by

$
\large
\begin{align}
<x>\ = \int_{0}^{100} |ψ(x)|^2 x dx.
\end{align}
$

The mean should be equal to 79$\pm$-1.

Hint: use the procedure for creating the Hamiltonian matrix from tasks 1, 2. This will reduce the amount of coding required.

I have implemented both the explicit euler and the spectral method for most of the problems. I have decided to leave both in, for my future refrencing of the methods, if I will ever need them.

In [ ]:
### Constants for task 3 and 4
x0     = 15
v      = 16
sigma2 = 5
tmax   = 4

In [ ]:
from scipy.integrate import quad
from scipy.integrate import simps

In [ ]:
def task3_gaussian(x):
    """Schrodinger wavepacket, centered at x0=15, with standard deviation of sigma2=5 and velocity of v=16.
    Takes in an array of x values and for each entry returns an array of complex numbers associated with those x values."""
    return np.exp(-(x-x0)**2/(2*sigma2) + 1j*v*x/2)

def task3_probability_function(x):
    """Probability distribution function.
    Takes an array of values, returns another array"""
    return np.exp(-(x-x0)**2/sigma2)

In [ ]:
# Normalize the gaussian:
normConstSq = quad(task3_probability_function, 0, 100)[0]
normConst = np.sqrt(normConstSq)
print(1/normConst)

In [ ]:
def task3_gaussian_norm(x):
    """Normalized form of the task3_gaussian function. 
    Takes in an array, returns the wavefunction numbers associated with every entry in the array. Returns an array"""
    return 1/normConst * task3_gaussian(x)

## Task 3 plot and result

In the cell(s) below, do the following:
1. Solve equation (1) for t=0...4 and plot $|\psi(x,t=0)|^2$ and $|\psi(x,t=tmax)|^2$ as a function of x.
2. Compute the mean of the final position and print the value. The correct result should be between 78 and 80.

Don't forget to nomalize your Gaussian initial condition so that the total probability $\int_0^L |\psi(x,t=0)|^2 dx=1$. 

You do not have to wrap up the calculation in a function "task3()", but doing so may help to reduce the amount of coding for tasks 4-6.

In [ ]:
# dx is stored as my_dx

# Choose small enough dt
dt = my_dx**2/2
x00 = 0
x0N = 100
N = int((x0N - x00) / my_dx)
x = np.linspace(x00, x0N, N)
tmax = 4
nmax = int(tmax/dt)

In [ ]:
def task3_explicitE():
    # Initial position
    psi0 = task3_gaussian_norm(x)
    
    hamiltonian = make_Laplacian(N, my_dx)
    
    psin = psi0.copy()
    
    A = identity(N) + 1/2 * 1j * hamiltonian * dt
    b = identity(N) - 1/2 * 1j * hamiltonian * dt
    
    # Iterate
    for n in range(nmax):
        RHS = b.dot(psin)
        psin = spsolve(A, RHS)
        
    # Calculate the position probabilites
    p0 = psi0 * psi0.conjugate()
    pn = psin * psin.conjugate()
    
    mean = simps(x * pn, x)
    print(f'Mean of the final position is {mean.real}') # Imaginary part is 0 but it still stored as 0, doing mean.real displays nicer
    
    plt.plot(x, p0.real, label='t=0') # .real to get rid of the annoying error message
    plt.plot(x, pn.real, label=f't={tmax}')
    plt.title('Probability of a particle being at a specific position for $t=0$ and for $t=4$')
    plt.legend()
    plt.xlabel('x')
    plt.ylabel('Probability distribution')
    plt.show()

Uncomment the line task3_explicitE() to see the explicit euler method. The spectral method is faster and that is why I decided to use that one.

In [ ]:
t1 = time.time()
#task3_explicitE()
t2 = time.time()
print(f'Time to solution is {t2 - t1} seconds')

In [ ]:
from matplotlib import animation
from IPython.display import HTML

In [ ]:
def make_animation_task3():
    fig, ax = plt.subplots()
    ax.set_xlim(0, 100)
    ax.set_ylim(0, 0.25)
    line, = ax.plot([], [], lw=2)
    
    hamiltonian = make_Laplacian(N, my_dx)
    psi0 = task3_gaussian_norm(x)
    psin = psi0.copy()
    
    A = identity(N) + 1/2 * 1j * hamiltonian * dt
    b = identity(N) - 1/2 * 1j * hamiltonian * dt
    
    ps = []
    
    for n in range(nmax):
        RHS = b.dot(psin)
        psin = spsolve(A, RHS)
        ps.append(np.abs(psin)**2)
    
    def init():
        line.set_data(x, psi0)
        return (line,)
    
    def animate(i):
        line.set_data(x, ps[10*i])
        return (line,)
    
    anim = animation.FuncAnimation(fig, animate, init_func=init,
                                   frames=int(nmax/10), interval=20, blit=True)

    return anim
    
# Uncomment the next line of code to produce an animation
HTML(make_animation_task3().to_jshtml())

In [ ]:
def task3_spectral():
    # Spectral method
    
    # Create the hamiltonian
    hamiltonian = make_Laplacian(N, my_dx)
    hamiltonian = hamiltonian.toarray()
    
    u0 = task3_gaussian_norm(x)
    
    # Get eigenvalues
    lam, phi = eigh_tridiagonal(np.diagonal(hamiltonian), np.diagonal(hamiltonian, 1))
    a = phi.T.dot(u0)
    
    u = np.zeros(N, dtype=np.complex128)
    for i in range(lam.size):
        u += a[i] * np.exp(-1j * lam[i] * tmax) * phi[:, i]
    
    p0 = u0 * np.conjugate(u0)
    pn = u * np.conjugate(u)
    
    mean = simps(x * pn, x)
    print(f'Mean of the final position is {mean.real}')
    
    plt.plot(x, p0.real, label='t=0') # p0.real to get rid of the annoying error message for casting complex values
    plt.plot(x, pn.real, label=f't={tmax}')
    plt.title('Probability of a particle being at a specific position for $t=0$ and for $t=4$')
    plt.legend()
    plt.xlabel('x')
    plt.ylabel('Probability')
    plt.show()

t1 = time.time()
task3_spectral()
t2 = time.time()
print(f'Time to solution is {t2 - t1} seconds')

# Task 4 (10p)

Now repeat task 3 for a potential made up of regularly spaced wells such that

V=70 for |x-i| < 0.25 where i=0,1,...,100,

and V=0 elsewhere.

Plot the potential. It should be $V=70$ for $x=0...0.25$, $V=0$ for $x=0.25...0.75$, $V=70$ for $x=0.75...1.25$, and so on.

Determine the mean of $|\psi(x,t)|^2$ as before, with accuracy $\pm$1.

Make sure that dx and dt are sufficiently small to achieve this accuracy!

In the cell below, create the potential and plot it over the range [0, 3].

In [ ]:
# Create the potential
V = []
for oneX in x:
    count = 0
    for i in range(100):
        if np.abs(oneX-i) < 0.25:
            count += 1
            break
    if count == 1:
        V.append(70)
    else:
        V.append(0)
        
fig, ax = plt.subplots()
ax.set_xlim(0, 3)
ax.plot(x, V)
plt.xlabel('Position')
plt.ylabel('Potential')
plt.show()

# Task 4 continued

In the cell below, repeat task 3 with the new potential.

Again, I have implement both the explicit euler and the spectral method, but have decided to only let the spectral one run because it is faster.

In [ ]:
### Constants for task 3 and 4
x0     = 15
v      = 16
sigma2 = 5
tmax   = 4

# Choose small enough dt
dt = my_dx**2/2
x00 = 0
x0N = 100
N = int((x0N - x00) / my_dx)
x = np.linspace(x00, x0N, N)
tmax = 4
nmax = int(tmax/dt)

In [ ]:
def create_Hamiltonian(n, dx, potential):
    """Creates a laplacian plus the potential operator.
    Parameters are: 
    n - size of the matrix
    dx - spacing between consecutive points
    potential - array of values of the potential at the points we create the laplacian, has to be of length n
    
    Function returns a hamiltonian matrix of size n times n"""
    
    # Create the Laplace operator
    diagOne = np.ones(n-1)
    diagZero = -2*np.ones(n)
    # Manually change
    diagZero[0] = -1
    diagZero[-1] = -1
    # Add the potential values
    for i in range(n):
        diagZero[i] -= potential[i] * dx**2 # Changed to potential - used to be V[i]
    
    diagMinusOne = np.ones(n-1)
    hamilton = -1/(dx**2) * diags([diagMinusOne, diagZero, diagOne], [-1,0,1])
    
    return hamilton

In [ ]:
print(create_Hamiltonian(5, 1, [0, 0, 0, 0, 0]).toarray())

In [ ]:
def task4_explicitE():
    # Initial position
    psi0 = task3_gaussian_norm(x)
    
    hamiltonian = create_Hamiltonian(N, my_dx, V)
    
    psin = psi0.copy()
    
    A = identity(N) + 1/2 * 1j * hamiltonian * dt
    b = identity(N) - 1/2 * 1j * hamiltonian * dt
    
    # Iterate
    for n in range(nmax):
        RHS = b.dot(psin)
        psin = spsolve(A, RHS)
        
    # Calculate the position probabilites
    p0 = psi0 * psi0.conjugate()
    pn = psin * psin.conjugate()
    
    mean = simps(x * pn, x)
    print(f'Mean of the final position is {mean.real}')
    
    plt.plot(x, p0.real, label='t=0')
    plt.plot(x, pn.real, label=f't={tmax}')
    plt.title('Probability of a particle being at a specific position for $t=0$ and for $t=4$')
    plt.legend()
    plt.xlabel('x')
    plt.ylabel('Probability distribution')
    plt.show()

    
# Uncomment the line task4_explicitE() to see the result of the explicit euler method
t1 = time.time()
#task4_explicitE()
t2 = time.time()
print(f'Time to solution is {t2 - t1} seconds')

In [ ]:
def make_animation_task4():
    fig, ax = plt.subplots()
    ax.set_xlim(0, 100)
    ax.set_ylim(0, 0.25)
    line, = ax.plot([], [], lw=2)
    
    hamiltonian = create_Hamiltonian(N, my_dx, V)
    psi0 = task3_gaussian_norm(x)
    psin = psi0.copy()
    
    A = identity(N) + 1/2 * 1j * hamiltonian * dt
    b = identity(N) - 1/2 * 1j * hamiltonian * dt
    
    ps = []
    
    for n in range(nmax):
        RHS = b.dot(psin)
        psin = spsolve(A, RHS)
        ps.append(np.abs(psin)**2)
    
    def init():
        line.set_data(x, psi0)
        return (line,)
    
    def animate(i):
        line.set_data(x, ps[15*i])
        return (line,)
    
    anim = animation.FuncAnimation(fig, animate, init_func=init,
                                   frames=int(nmax/15), interval=20, blit=True)

    return anim
    
# Uncomment the next line to make an animation
HTML(make_animation_task4().to_jshtml())

In [ ]:
def task4_spectral():
    # Spectral method
    
    # Create the hamiltonian
    hamiltonian = create_Hamiltonian(N, my_dx, V)
    hamiltonian = hamiltonian.toarray()
    
    u0 = task3_gaussian_norm(x)
    
    # Get eigenvalues
    lam, phi = eigh_tridiagonal(np.diagonal(hamiltonian), np.diagonal(hamiltonian, 1))
    a = phi.T.dot(u0)
    
    u = np.zeros(N, dtype=np.complex128)
    for i in range(lam.size):
        u += a[i] * np.exp(-1j * lam[i] * tmax) * phi[:, i]
    
    p0 = u0 * np.conjugate(u0)
    pn = u * np.conjugate(u)
    
    mean = simps(x * pn, x)
    print(f'Mean of the final position is {mean.real}')
    
    plt.plot(x, p0.real, label='t=0')
    plt.plot(x, pn.real, label=f't={tmax}')
    plt.title('Probability of a particle being at a specific position for $t=0$ and for $t=4$')
    plt.legend()
    plt.xlabel('x')
    plt.ylabel('Probability')
    plt.show()
    
t1 = time.time()
task4_spectral()
t2 = time.time()
print(f'Time to solution is {t2 - t1} seconds')

# Task 5 (15p)

Calculate the probability $P_{1/2}$ of the particle moving through the point x=L/2 by integrating the probability current 

$
\large
\begin{align}
j = (\psi^*  \frac{\partial \psi}{\partial x} -
\psi \frac{\partial \psi^*}{\partial x})(x=L/2)
\end{align}
$

over time, for t=0...4. The probability can deviate from the true value by no more than $\pm$0.01 (hint: the correct value is between 0.5 and 1).

In the cell below, calculate the probability and print your answer.

As usual, the derivative is approximated by a finite difference. The derivative at $x = L/2$ looks like this: 
$\begin{align}
\frac{\partial \psi} {\partial x} = \frac{\psi (L/2 + dx) - \psi(L/2)}{dx}
\end{align}$. 
Approximate both the derivative and the conjugate derivative, store the values for each t and calculate the integral.

For this task the spectral method again outperforms the explicit euler one.

In [ ]:
x0     = 15
v      = 16
sigma2 = 5
tmax   = 4

# Choose small enough dt
dt = my_dx**2/2
x00 = 0
x0N = 100
N = int((x0N - x00) / my_dx)
x = np.linspace(x00, x0N, N)
tmax = 4
nmax = int(tmax/dt)

In [ ]:
def get_current_prob_density_at_halfL(psi_halfL, psi_halfL_dx):
    """This function takes in the values of psi at L/2 and L/2+dx and returns the approximated probability current."""
    
    currentj = np.conjugate(psi_halfL) * (psi_halfL_dx - psi_halfL)/my_dx - psi_halfL * np.conjugate((psi_halfL_dx - psi_halfL) / my_dx)
    currentj = -1j * currentj
    return currentj

In [ ]:
def task5_spectral():
    times = np.linspace(0, tmax, int(nmax/10))
    # Use the same potential as for the previous task
    # It is stored as V
    
    # Create the hamiltonian
    hamiltonian = create_Hamiltonian(N, my_dx, V)
    hamiltonian = hamiltonian.toarray()
    
    u0 = task3_gaussian_norm(x)
    
    # Get eigenvalues
    lam, phi = eigh_tridiagonal(np.diagonal(hamiltonian), np.diagonal(hamiltonian, 1), select='i', select_range=(0, 600))
    a = phi.T.dot(u0)
    
    js = []
    index = int(N/2)
    
    for time in times:
        psi_halfL = sum(a[i] * np.exp(-1j * lam[i] * time) * phi[index, i] for i in range(lam.size))
        psi_halfL_dx = sum(a[i] * np.exp(-1j * lam[i] * time) * phi[index + 1, i] for i in range(lam.size))
            
        js.append(get_current_prob_density_at_halfL(psi_halfL, psi_halfL_dx))
        
    probability = simps(js, times)
    print(f'Probability of the partice moving through the point x = L/2 is {probability.real}')

t1 = time.time()
task5_spectral()
t2 = time.time()

print(f'Time take to complete the calculations is {t2 - t1} seconds')

In [ ]:
def task5_explicitE():
    times = np.linspace(0, tmax, nmax)
    # Initial position
    psi0 = task3_gaussian_norm(x)
    
    hamiltonian = create_Hamiltonian(N, my_dx, V)
    
    psin = psi0.copy()
    
    A = identity(N) + 1/2 * 1j * hamiltonian * dt
    b = identity(N) - 1/2 * 1j * hamiltonian * dt
    
    js = []
    index = int(N/2)
    
    # Iterate
    for n in range(nmax):
        RHS = b.dot(psin)
        psin = spsolve(A, RHS)
        
        js.append(get_current_prob_density_at_halfL(psin[index], psin[index + 1]))
        
    probability = simps(js, times)
    print(f'Probability of the partice moving through the point x = L/2 is {probability.real}')
    
# Uncomment the line bellow to see the explicit euler method result
t1 = time.time()
#task5_explicitE()
t2 = time.time()

print(f'Time take to complete the calculations is {t2 - t1} seconds')

# Task 6 (15p)

Plot the probability $P_{1/2}$ as a function of particle energy E=0...100, for at least 100 equally-spaced values from this range. All $P_{1/2}$ values should be within $\pm$0.01 of the true values. Use the formula 

$
\large
\begin{align}
E = \frac{1}{4} v^2
\end{align}
$

to convert between energy and velocity (valid for Eqs. (1,2)).

Bonus question: can you explain why the plot looks like this?

Plot the probability in the cell below.

In [ ]:
x0     = 15
sigma2 = 5
tmax   = 4

# Choose small enough dt
dt = my_dx**2
x00 = 0
x0N = 100
N = int((x0N - x00) / my_dx)
x = np.linspace(x00, x0N, N)
tmax = 4
nmax = int(tmax/dt)

In [ ]:
def energy_gaussian_task6(x, E):
    """Function of the same form as the usuall gaussian, the energy is a parameter in this case and it determines the velocity"""
    v = np.sqrt(4 * E)
    return 1/normConst * np.exp(-(x-x0)**2/(2*sigma2) + 1j*v*x/2)

In [ ]:
def task6_explicitE():
    energies = np.linspace(0, 100, 100)
    times = np.linspace(0, tmax, nmax)
    probabilities = []
    
    for energy in energies:
        print(energy)
        # Initial position
        psi0 = energy_gaussian_task6(x, energy)
    
        hamiltonian = create_Hamiltonian(N, my_dx, V)
        
        psin = psi0.copy()
        
        A = identity(N) + 1/2 * 1j * hamiltonian * dt
        b = identity(N) - 1/2 * 1j * hamiltonian * dt
        
        js = []
        index = int(N/2)
        
        # Iterate
        for n in range(nmax):
            RHS = b.dot(psin)
            psin = spsolve(A, RHS)
            
            js.append(get_current_prob_density_at_halfL(psin[index], psin[index + 1]))
        
        probability = simps(js, times)
        probabilities.append(probability.real) # complex part is 0 anyways, and .real gets rid of the error message
        
    plt.plot(energies, probabilities)
    plt.title('Probability of a particle passing through the point $x=L/2$ vs the starting energy of the particle')
    plt.xlabel('Energy')
    plt.ylabel('Probability of passing through x = L/2')
    plt.show()

# This method takes way too long to run, and I have implemented the same problem using the spectral method below.
# I left this in because it does return very accurate results, even though takes a very long time to run.
# I used it to compare the results of the spectral method to see if the accuracy is good enough
t1 = time.time()
#task6_explicitE()
t2 = time.time()

print(f'Time take to complete the calculations is {t2 - t1}')

In [ ]:
def task6_spectral():
    energies = np.linspace(0, 100, 100)
    times = np.linspace(0, tmax, int(nmax/100))
    
    probabilities = []
    
    # Use the same potential as for the previous task
    # It is stored as V
        
    # Create the hamiltonian
    hamiltonian = create_Hamiltonian(N, my_dx, V)
    hamiltonian = hamiltonian.toarray()
    # Get eigenvalues
    lam, phi = eigh_tridiagonal(np.diagonal(hamiltonian), np.diagonal(hamiltonian, 1), select='i', select_range=(0, 600))

    for energy in energies:
        
        # Get the specific starting gaussian distribution
        u0 = energy_gaussian_task6(x, energy)
        
        a = phi.T.dot(u0)
        
        js = []
        index = int(N/2)
        
        for time in times:
            psi_halfL = sum(a[i] * np.exp(-1j * lam[i] * time) * phi[index, i] for i in range(lam.size))
            psi_halfL_dx = sum(a[i] * np.exp(-1j * lam[i] * time) * phi[index + 1, i] for i in range(lam.size))
            
            js.append(get_current_prob_density_at_halfL(psi_halfL, psi_halfL_dx))
        
        probability = simps(js, times)
        probabilities.append(probability.real) # complex part is 0 anyway, and .real gets rid of the error message
        
    plt.plot(energies, probabilities)
    plt.title('Probability of a particle passing through the point $x=L/2$ vs the starting energy of the particle')
    plt.xlabel('Energy')
    plt.ylabel('Probability of passing through x = L/2')
    plt.show()
    
time1 = time.time()
task6_spectral()
time2 = time.time()
print(f'Time taken for this task to complete is {time2 - time1} seconds')

My best guess is that the reason the graph looks this way is:
- The initial low probability of passing through the midpoint is because the initial energy and therefore velocity are both too low for the particle to pass the barriers and have enough time to reach the midpoint
- The energy increases enough for the particle to effectively pass the bariers and pass through the midpoint - thats where the probability increases
- The probability then falls off because the particle is fast enough to reach the midpoint on its way back, after it had reflected of the right boundary. Because we are only calculating the probability of passing through the midpoint from left to right, this decreases the probability because it is counted as negative current
- The probability then again starts increasing, my bet is that the particle gets reflected 2 times and starts reaching the midpoint for the third time, this time from the left so the current is counted as positive


# Task 7 (10p)

Assume again the initial condition of equation (2) with v=16, and consider a disordered potential in which

V=70 for |x-i| < b$_i$ where i=0,1,...,100,

and b$_i$ is a random variable uniformly distributed on [0.125, 0.375].

Find the probability $P_{1/2}$ by averaging over 100 realizations of the random potential (must be accurate to $\pm$0.02). Plot the histogram of $P_{1/2}$. Comment on the value of $P_{1/2}$ compared with task 5.

In [ ]:
x0     = 15
v      = 16
sigma2 = 5
tmax   = 4

# Choose small enough dt
dt = my_dx**2/2
x00 = 0
x0N = 100
N = int((x0N - x00) / my_dx)
x = np.linspace(x00, x0N, N)
tmax = 4
nmax = int(tmax/dt)

In [ ]:
def disordered_potential():
    """This function creates and returns a disordered potential in a list. V[i] corresponds to the position x[i]"""
    bi = (0.375 - 0.125) * np.random.random_sample(100) + 0.125
    V = []
    for oneX in x:
        count = 0
        for i in range(100):
            if np.abs(oneX-i) < bi[i]:
                count += 1
                break
        if count == 1:
            V.append(70)
        else:
            V.append(0)
    return V

V = disordered_potential()

fig, ax = plt.subplots()
ax.set_xlim(0, 3)
ax.plot(x, V)
plt.xlabel('Position')
plt.ylabel('Potential')
plt.show()

In [ ]:
def task7():
    probabilities = []
    
    for i in range(100):
        V = disordered_potential()
        
        times = np.linspace(0, tmax, int(nmax/100))
        
        # Create the hamiltonian
        hamiltonian = create_Hamiltonian(N, my_dx, V).toarray()
        
        u0 = task3_gaussian_norm(x)
        
        # Get eigenvalues
        lam, phi = eigh_tridiagonal(np.diagonal(hamiltonian), np.diagonal(hamiltonian, 1), select='i', select_range=(200, 600))
        a = phi.T.dot(u0)
        
        js = []
        index = int(N/2)
        
        for time in times:
            psi_halfL = sum(a[i] * np.exp(-1j * lam[i] * time) * phi[index, i] for i in range(lam.size))
            psi_halfL_dx = sum(a[i] * np.exp(-1j * lam[i] * time) * phi[index + 1, i] for i in range(lam.size))
            
            js.append(get_current_prob_density_at_halfL(psi_halfL, psi_halfL_dx))
        
        probability = simps(js, times)
        probabilities.append(probability.real)
    
    print(f'Average probability of passing through the midpoint of a disordered potential is {np.average(probabilities).real}')
    hist, bins, p = plt.hist(probabilities, bins=100)
    plt.title('Probability of a particle passing through the midpoint for a 100 realizations of the disordered potential')
    plt.xlabel('Probability of passing through the midpoint')
    plt.ylabel('Number of occurences')
    plt.show()


    
t1 = time.time()
task7()
t2 = time.time()

print(f'Time taken for this exercise is {t2 - t1} seconds')

Even though the magnitude of the potential remained the same, the fact that it is very disordered this time causes the particle to stay localized and not move that much. That is why the probability of passing through the midpoint became very low and is much lower than the same probability in task 5 with an ordered potential. Looking at the animation of a single realization of the unordered potential, we can see that the particle tends to stay traped in pockets of the potential barriers.

In [ ]:
def make_animation_task7():
    # Animates the schrodinger wavefunction evolution for one randomized potential
    fig, ax = plt.subplots()
    ax.set_xlim(0, 100)
    ax.set_ylim(0, 0.25)
    line, = ax.plot([], [], lw=2)
    
    V = disordered_potential()
    
    hamiltonian = create_Hamiltonian(N, my_dx, V)
    psi0 = task3_gaussian_norm(x)
    psin = psi0.copy()
    
    A = identity(N) + 1/2 * 1j * hamiltonian * dt
    b = identity(N) - 1/2 * 1j * hamiltonian * dt
    
    ps = []
    
    for n in range(nmax):
        RHS = b.dot(psin)
        psin = spsolve(A, RHS)
        ps.append(np.abs(psin)**2)
    
    def init():
        line.set_data(x, psi0)
        return (line,)
    
    def animate(i):
        line.set_data(x, ps[15*i])
        return (line,)
    
    anim = animation.FuncAnimation(fig, animate, init_func=init,
                                   frames=int(nmax/15), interval=20, blit=True)

    return anim
    
# Uncomment the next line to make an animation
HTML(make_animation_task4().to_jshtml())